### Model Inference

it was observerd that the model is not understanding anything

In [ ]:
import torch
import os
import json
import pickle
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from utils import build_vocab, collate_fn, load_json
from dataset import FinanceDataset
from model import Encoder, Decoder, Seq2Seq
from wandb_config import init_wandb

train_data = load_json('/content/train.json')
vocab = build_vocab(train_data)

with open('vocab.pkl', 'wb') as f: 
    pickle.dump(vocab, f)




embedding_dim = 50
hidden_dim = 32
max_len = 128


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


encoder = Encoder(len(vocab), embedding_dim, hidden_dim).to(device)
decoder = Decoder(len(vocab), embedding_dim, hidden_dim, len(vocab)).to(device)
model = Seq2Seq(encoder, decoder, device).to(device)


model.load_state_dict(torch.load("/content/seq2seq_epoch_12.pth", map_location=device))
model.eval()


def predict(input_text, max_output_len=50):
    with torch.no_grad():
        src_tensor = encode_sentence(input_text, vocab, max_len=max_len).unsqueeze(1).to(device)
        hidden, cell = model.encoder(src_tensor)

        input_token = torch.tensor([vocab['<sos>']]).to(device)
        outputs = []

        for _ in range(max_output_len):
            output, hidden, cell = model.decoder(input_token, hidden, cell)
            top1 = output.argmax(1).item()

            if vocab.get_itos()[top1] == '<eos>':
                break

            outputs.append(vocab.get_itos()[top1])
            input_token = torch.tensor([top1]).to(device)

    return ' '.join(outputs)

# sample trial 1
question = "whaat is loan?"
response = predict(question)
print("User:", question)
print("FinanceBot:", response)


User: whaat is loan?
FinanceBot: i ' the the , and the . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .


In [17]:
##sample trial 2

question = "Should I invest in my house, when it's in my wife's name?"
response = predict(question)
print("User:", question)
print("FinanceBot:", response)

User: Should I invest in my house, when it's in my wife's name?
FinanceBot: i ' s a to the . . . you can ' t have to the . . . you can ' t have to the . . . you can ' t have to the . . . you can ' t have to the . . . you


In [18]:
## sample trial 3

question = "hello! what is your name?"
response = predict(question)
print("User:", question)
print("FinanceBot:", response)

User: hello! what is your name?
FinanceBot: i ' s a to the . . . the . . . . . . the . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
